In [1]:
from sklearn import preprocessing
from collections import namedtuple
from collections import defaultdict
from collections import Counter
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
from numba import jit
%matplotlib inline

import os
# unzip all datafile in <PROJECT_ROOT>/data_format1/*
# dataset is huge. So this may take some time ...
project_root = "." # this di
data_folder =  "data"

In [ ]:
data_test = pd.read_csv(os.path.join(project_root, data_folder ,"test_format1.csv"))
data_usr = pd.read_csv(os.path.join(project_root, data_folder ,"user_info_AGNormal_dv.csv"))
agg_merchant = pd.read_csv(os.path.join(project_root, data_folder ,"agg_merchant.csv"))
agg_user = pd.read_csv(os.path.join(project_root, data_folder ,"agg_user.csv"))


In [ ]:
data_train = pd.read_csv(os.path.join(project_root, data_folder ,"train_format1.csv"))
merchant_profile = pd.read_csv(os.path.join(project_root, data_folder ,"merchant_profile_dc_dv.csv"))
data_train_embedding = pd.merge(data_train, merchant_profile, how='left', on=['merchant_id'])
data_train_embedding = pd.merge(data_train_embedding, data_usr, how='left', on=['user_id'])
data_train_embedding = pd.merge(data_train_embedding, agg_merchant, how='left', on=['merchant_id'])
data_train_embedding = pd.merge(data_train_embedding, agg_user, how='left', on=['user_id'])
data_train_embedding.drop(['merchant_id','user_id','Unnamed: 0_x','Unnamed: 0_y'],axis=1,inplace=True)

In [ ]:
data_test = pd.read_csv(os.path.join(project_root, data_folder ,"test_format1.csv"))
data_test_embedding = pd.merge(data_test, merchant_profile, how='left', on=['merchant_id'])
data_test_embedding = pd.merge(data_test_embedding, data_usr, how='left', on=['user_id'])
data_test_embedding = pd.merge(data_test_embedding, agg_merchant, how='left', on=['merchant_id'])
data_test_embedding = pd.merge(data_test_embedding, agg_user, how='left', on=['user_id'])
data_test_embedding.drop(['merchant_id','user_id','Unnamed: 0_x','Unnamed: 0_y'],axis=1,inplace=True)
data_test_embedding.drop(['prob'],axis=1,inplace=True)
data_test_embedding.fillna(0,axis=1,inplace=True)

In [ ]:
data_train_embedding.count()

In [ ]:
train = np.array(data_train_embedding).T
print(train)
test = np.array(data_test_embedding)
train_y = train[:][0]
train_X = train[:][1:].T
train_X.shape
#split data into x y
# dtrain = xgb.DMatrix(train)
# dtest = xgb.DMatrix(test)

In [ ]:
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from tqdm import tqdm
from sklearn.metrics import roc_curve,auc

kf = KFold(train_X.shape[0], n_folds=10, shuffle=True)
clf = xgb.XGBClassifier(seed=7,silent=False,n_jobs =-1)

# from sklearn.ensemble import GradientBoostingRegressor
# clf=GradientBoostingRegressor(
#   loss='ls', learning_rate=0.1, n_estimators=100, subsample=1, min_samples_split=2, min_samples_leaf=1, 
#     max_depth=7, init=None, random_state=None, max_features=None, alpha=0.9, verbose=1, max_leaf_nodes=None, 
#     warm_start=False,
# )

# clf = RandomForestClassifier(max_depth=5, random_state=0,n_jobs =-1)

# from sklearn import svm
# clf = svm.SVC(kernel='linear', C=1)

# kf = tqdm(kf, ascii = True, ncols = 88)
# tqdm is a progress bar function.
# kf.set_description("acc %0.03f %%"%(0*100))
for epoch , (train_index, test_index) in enumerate(kf):
    X_train, X_test = train_X[train_index], train_X[test_index]
    y_train, y_test = train_y[train_index], train_y[test_index]
    clf.fit(X_train,y_train)
    predictions=clf.predict_proba(X_test)#每一类的概率
    false_positive_rate, recall, thresholds = roc_curve(y_test, predictions[:, 1])
    roc_auc=auc(false_positive_rate,recall)
#     kf.set_description("acc %0.03f %%"%(roc_auc*100))
    print("=> finished epoch %d, val acc %0.03f %%"%(epoch,roc_auc*100))
    

In [ ]:
clf.fit(train_X,train_y)
predictions=clf.predict_proba(train_X)#每一类的概率
false_positive_rate, recall, thresholds = roc_curve(train_y, predictions[:, 1])
roc_auc=auc(false_positive_rate,recall)
print("=> after all, train acc %0.03f %%"%(roc_auc*100))

In [ ]:
pred = (clf.predict_proba(test).T[1].T)
pred

In [ ]:
data_test['prob'] = DataFrame(pred)
data_test.set_index(['user_id', 'merchant_id'], inplace=True)

In [ ]:
data_test.to_csv(os.path.join(project_root, 'submit', 'TeYeT_ass' ,"xgb_macrTT_mdcTT_uacrTT_udv_mdv_maggTT_uaggTT_mr.csv"))
# uacr - merchant action count rate monthly(all) 
# macr - merchant action count rate monthly(all) 
# mr - merchant repeat rate
# mdc - merchant day count
# TT - total

In [ ]:
data_test